# Pregunta 2 Tarea 2

## GF5013 - Metodos Inversos Aplicados a la Geofisica
## Primavera 2019
Prof. Francisco Hernán Ortega Culaciati 

ortega.francisco@uchile.cl

Departamento de Geofísica - FCFM

Universidad de Chile
Octubre de 2019

Nombre Alumno: XXXX YYYY

In [ ]:
import sys, os
pathTarea2 = '../'
pathPaquetesPython = pathTarea2 + '/PaquetesPython'
TomoDataFilename = pathTarea2 + '/datosTomo/datosTomografia.txt'

sys.path.append(os.path.abspath(pathPaquetesPython))


In [ ]:
import matplotlib
%matplotlib notebook
# descomentar esto es para hacer los plots afuera del notebook
#%matplotlib tk  


import matplotlib.pyplot as plt
import numpy as NP

from Tarea2.TomoData import readTravelTimeData as readTTdata
from Tarea2 import TomographyModel as Tmodel
import Tarea2
import GF5013

## Definir la malla del problema

In [ ]:
Xmin = 0
Xmax = 100
Nx = 9
Zmin = -200
Zmax = 0
Nz = 15

mesh = Tmodel.define_Tomography_RectangularMesh(Xmin = Xmin, Xmax = Xmax, Nx = Nx, Zmin = Zmin, Zmax = Zmax, Nz = Nz)


## Formar matriz de diseño del problema de tomografía lineal

In [ ]:
# LEER DATOS TOMOGRAFIA
TTdata = readTTdata(TomoDataFilename)
# Coordenadas Fuentes
xF = TTdata['xF']
zF = TTdata['zF']
# Coordenadas Receptores
xR = TTdata['xR']
zR = TTdata['zR']
# Tiempo de Viaje
TV = TTdata['TV']

# formar matriz G y vector de datos G
G, d, rays = Tmodel.formar_G(xF, zF, xR, zR, TV, mesh, fractional_precision=0.01)


In [ ]:
# generar datos sinteticos
d_syn, m_syn =  Tmodel.calcCheckerboardSynthetic(mesh, G, shift=[16.156, 16.156], scale=33.333)
# dibujar la malla con el modelo sintético
F = 1/m_syn # se grafica la velocidad ( no la lentitud)
fig = plt.figure(1)
fig.clear()
ax = plt.subplot(111)
fig, ax = Tarea2.Mesh.plotRectMeshField(F, mesh, fig, ax , colorlim = None, \
                                 title = "Modelo Sintético",\
                                 colorbar_label = 'Velocity ', xlabel = 'X [m]',\
                                 ylabel = 'Y [m]', colormap = 'gray')
fig.show()

## P2 (a) Inversion no regularizada sin ruido


In [ ]:
m_MCS_no_ruido = GF5013.optim.MinCuad.MinCuadSimple(G,d_syn)

In [ ]:
# (a) dibujar la malla con el modelo estimado con los datos SIN ruido
F = 1/m_MCS_no_ruido['m'] # se grafica la velocidad ( no la lentitud)
colorlim = None # [3,5]
fig = plt.figure(2)
fig.clear()
ax = plt.subplot(111)
fig, ax = Tarea2.Mesh.plotRectMeshField(F, mesh, fig, ax , colorlim = colorlim, \
                                 title = "Modelo Sintético",\
                                 colorbar_label = 'Velocity ', xlabel = 'X [m]', ylabel = 'Z [m]', colormap = 'gray')
fig.show()

Comentarios Aquí: 

## P2 (b) Inversión no regularizada CON ruido
## Se varía la amplitud del ruido para una misma realización del ruido aleatorio.

La idea es comparar como varía la solución sin cambiar la realización del ruido. 
Por esto primero se crea un vector de ruido iid con varianza = 1 y luego se amplifica este por las amplitudes mencionadas. 

Para poder tener control sobre los numeros aleatorios y poder hacer el experimento repetible, fijaremos la semilla del generador de numeros aleatorios en Python


In [ ]:
# semilla del generador de numeros aleatorios (use su fecha de nacimiento DDMMYYYY)
seed = 28101979
NP.random.seed(seed)

# calculo el ruido base
Ndata = len(d_syn)
ruidoBase = NP.random.randn(Ndata,1)

# Amplitudes del ruido a considerar en [ms]
AmplitudRuido = [0, 0.005, 0.02, 0.04] # notar que la amplitud del ruido es mucho menor que los tiempos
                                        # de viaje medidos (TV varía entre 20 y 40 ms)
    


In [ ]:
# realizar las inversiones con diferentes amplitudes de la realización del ruido. 
m_MCS_ruido = []
for A in AmplitudRuido:
    d_ruido = d_syn + A * ruidoBase
    m_MCS_ruido.append( GF5013.optim.MinCuad.MinCuadSimple(G, d_ruido)['m'] )
    

In [ ]:
# graficar las 4 soluciones
fig = plt.figure(3)
fig.clear()
fig.set_size_inches(9.8, 10.0, forward = True)


for i in range(0, len(AmplitudRuido)):
    A = AmplitudRuido[i]
    m_est = m_MCS_ruido[i]
    Vel_est = 1/m_est # se grafica la velocidad ( no la lentitud)
    colorlim = None #[3,5]
    
    ax = plt.subplot(2,2,i+1)
    
    ylabel = 'Z [m]'
  
    fig, ax = Tarea2.Mesh.plotRectMeshField(Vel_est, mesh, fig, ax , colorlim = colorlim, \
                                 title = "Amp. Ruido = {:.4f}".format(A),\
                                 colorbar_label = ' Estimated Velocity ', xlabel = 'X [m]',\
                                 ylabel = ylabel, colormap = 'gray', colorbar_orientation = 'vertical')


fig.show()

## Comentarios: 

# -----------------------------------------------------
## P3.c Inversión no regularizada CON ruido
## Se mantiene constante la amplitud del ruido en A = 0.02 pero se varía la realización del ruido aleatorio

In [ ]:
# realizar las inversiones con diferentes amplitudes de la realización del ruido. 
m_MCS_ruidoVar = []
A = 0.02
# se reinicia el generador de numeros aleatorios de manera que la primera solución es igual a la de la 
# parte anterior con A = 0.02
NP.random.seed(seed)
# Numero de inversiones:
Ninv = 4

for i in range(0,Ninv):
    d_ruido = d_syn + A * NP.random.randn(Ndata, 1)
    m_MCS_ruidoVar.append( GF5013.optim.MinCuad.MinCuadSimple(G,d_ruido)['m'] )
    
    

In [ ]:
# graficar las 4 soluciones
fig = plt.figure(4)
fig.clear()
fig.set_size_inches(9.8, 10.0, forward = True)

colorlim = None #[MinVel, MaxVel]


for i in range(0, Ninv):
    A = A
    m_est = m_MCS_ruidoVar[i]
    Vel_est = 1/m_est # se grafica la velocidad ( no la lentitud)

    
    ax = plt.subplot(2,2,i+1)
    if i == 0:
        ylabel = 'Z [m]'
    else:
        ylabel = ''
    fig, ax = Tarea2.Mesh.plotRectMeshField(Vel_est, mesh, fig, ax , colorlim = colorlim, \
                                 title = "Amp. Ruido = {:.4f}".format(A),\
                                 colorbar_label = ' Estimated Velocity ', xlabel = 'X [m]',\
                                 ylabel = ylabel, colormap = 'gray', colorbar_orientation = 'vertical')


fig.show()

## Comentarios Aquí:

# P2 (d) : Escriba su respuesta aquí

# P2 (e) : Escriba su respuesta aquí
